In [1]:
from aocd import get_data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import re
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt

In [2]:
f = open("test.txt", "r")
f = f.read()
data = f.split("\n") 

# Nr 15

In [92]:
f = open("test.txt", "r")
f = f.read()
data = f
data = data.split(",")

In [93]:
def hash(s, current):
    for x in list(s): 
        asc = ord(x)
        current += asc
        current = current * 17
        current = current % 256
    return current

In [94]:
current = 0
sum([hash(sample, current) for sample in data])

497373

In [125]:
current = 0

memory = {x:{} for x in range(257)}

for m,step in enumerate(data):
    if  "=" in step:
        k, v = step.split("=")
    elif "-" in step:
        k, v = step.split("-")
    else:
        print("ISSUE")
    box = hash(k, current)
    if  "=" in step:
        memory[box][k] = int(v)
    elif "-" in step:
        if k in memory[box].keys():
            del memory[box][k]
    else:
        print(step)
        print("bricked. Wrong formatting")

In [126]:
count = 0
for x in memory.keys():
    if len(memory[x]) == 0: 
        pass
    else:
        for n,y in enumerate(memory[x]):
            count+= (x+1) * (n+1) * memory[x][y]
count

259356

# Nr 16

In [81]:
d = np.array([list(x) for x in data])
d = np.pad(d, pad_width=1, constant_values="X") 

In [113]:
def handle_interaction(obstacle, direction):
    if (obstacle == ".") or (obstacle == "X"):
        return direction, None
    if (obstacle == "|") and ((direction == "b") or (direction == "t")):
        return direction, None
    if (obstacle == "-") and ((direction == "l") or (direction == "r")):
        return direction, None
    if obstacle == "\\":
        if direction == "t":
            return "l", None
        if direction == "b":
            return "r", None
        if direction == "r":
            return "b", None
        if direction == "l":
            return "t", None
    if obstacle == "/":
        if direction == "t":
            return "r", None
        if direction == "b":
            return "l", None
        if direction == "r":
            return "t", None
        if direction == "l":
            return "b", None
    # split cases
    if (obstacle == "|") and ((direction == "l") or (direction == "r")):
        return "t", "b"
    if (obstacle == "-") and ((direction == "b") or (direction == "t")):
        return "l", "r"


def walk_single_path(d, p, direction, path_track, keep_out):
    new_beams = []
    current_p = p
    current_direction = direction
    path_track[current_p[0], current_p[1]] += 1
    # run as long as you need to end up in the border
    count = 1
    for x in range(10000):
        count += 1
        if current_direction == "r":
            current_p = current_p + [0, 1]
        elif current_direction == "l":
            current_p = current_p + [0, -1]
        elif current_direction == "t":
            current_p = current_p + [-1, 0]
        elif current_direction == "b":
            current_p = current_p + [1, 0]
        current_direction, split = handle_interaction(
            d[current_p[0], current_p[1]], current_direction
        )

        k = hash(current_p.tobytes())
        if k in keep_out:
            if current_direction in keep_out[k]:
                    break
            else:
                keep_out[k].append(current_direction)
        else:
            keep_out[k] = []
        if split:
            new_beams.append([current_p, split])
        if d[current_p[0], current_p[1]] == "X":
            break
        path_track[current_p[0], current_p[1]] += 1
    if x == 4999:
        print("Max reached...")

    return path_track, new_beams

In [114]:
def run_full_position(starting_position, direction):

    track = np.zeros(d.shape)
    stack = [(starting_position, direction)]
    keep_out = {}
    while len(stack)>0:
        sp, di = stack.pop()
        track, new_beams = walk_single_path(d,sp,di, track, keep_out)
        for x in new_beams:
                stack.append(x)
    return (track[1:-1, 1:-1] > 0).sum()

In [115]:
# a
starting_position = np.array([1,0])
direction = "r"    
run_full_position(starting_position, direction)

8034

In [120]:
#b
stack = []
for x in range(1,d.shape[0]-1):
    starting_position = np.array([x,0])
    direction = "r"    
    stack.append(run_full_position(starting_position, direction))
    starting_position = np.array([x,-1])
    direction = "l"    
    stack.append(run_full_position(starting_position, direction))
for x in range(1,d.shape[1]-1):
    starting_position = np.array([0,x])
    direction = "r"    
    stack.append(run_full_position(starting_position, direction))
    starting_position = np.array([-1,x])
    direction = "l"    
    stack.append(run_full_position(starting_position, direction))
max(stack)

8225

# Nr 17

In [238]:
# Failed attempt. Dynamic programming doesnt work properly...

def direction(p0,p1,p3,p4,hist, distance_map, q, marker, memory):

    new = d[p0, p1] + distance_map[p3, p4]

    if distance_map[p0, p1] > -1:
        if new < distance_map[p0, p1]:
            distance_map[p0, p1] = new
    else:
        distance_map[p0, p1] = new

    if (p0, p1, hist + [marker]) not in memory:
        if (len(set((hist + [marker])[-4:])) != 1) or (len(hist + [marker]) < 4) :
            q.append((p0, p1, hist + [marker]))

    return distance_map,q



def dynamic(d,distance_map, q,memory):
    for x in range(10000):
   # while len(q) > 0:
        p = q.pop(0)
        memory.append(p)
        # update left and top
        if (p[0] > 0) and (p[2][-1] !="d") :
            distance_map, q = direction(p[0]-1,p[1],p[0],p[1],p[2], distance_map, q, "u", memory)
        if (p[1] > 0) and (p[2][-1] !="r") :
            distance_map, q = direction(p[0],p[1]-1,p[0],p[1],p[2], distance_map, q, "l", memory)
        if (p[0] < (len(d)-1)) and (p[2][-1] !="u") :
            distance_map, q = direction(p[0]+1,p[1],p[0],p[1],p[2], distance_map, q, "d", memory)
        if (p[1] > (len(d[0])-1)) and (p[2][-1] !="l") :
            distance_map, q = direction(p[0],p[1]+1,p[0],p[1],p[2], distance_map, q, "r", memory)
    print(x)
    return distance_map, q

d =  np.array([list(x) for x in data]).astype(int)
d = np.flip(d).T
distance_map = np.zeros(d.shape)
distance_map[:,:] = -1
q = [(len(d)-1, len(d[0])-1, ["Start"])]
memory = []
distance_map[len(d)-1, len(d[0])-1] = d[len(d)-1, len(d[0])-1]

out, q = dynamic(d,distance_map, q, memory)
np.flip(out.T)



In [17]:
d =  np.array([list(x) for x in data]).astype(int)

# structure: (priority(current length), p0,p1,history[-4:])

q = [(2, [(0, 0)], np.array(["S"]))]


for x in range(200): 
    q.sort() 
    print(len(q))
    item = q.pop(0)
    p = item[1][-1]


    # move up
    if (p[0] > 0)  and not (np.all(item[-1][-3:] == "u")) and ((p[0]-1, p[1]) not in item[1]) :
        new = d[p[0]-1, p[1]] + item[0]
        q.append((new, item[1] + [(p[0]-1, p[1])], np.append(item[2], "u" )))
    # move left
    if (p[1] > 0)  and not (np.all(item[-1][-3:] == "l")) and ((p[0], p[1]-1) not in item[1]) :
        new = d[p[0], p[1]-1] + item[0]
        q.append((new, item[1] + [(p[0], p[1]-1)], np.append(item[2], "l" )))
    # move down
    if (p[0] < (len(d[0])-1))   and not (np.all(item[-1][-3:] == "d")) and ((p[0]+1, p[1]) not in item[1]) : 
        new = d[p[0]+1, p[1]] + item[0]
        if (p[0]+1 == len(d)) and (p[1] == len(d[0])):
            print(new)
            break
        else:
            q.append((new, item[1] + [(p[0]+1, p[1])], np.append(item[2], "d" )))
    # move  right
    if (p[1] <(len(d[1])-1))   and  not (np.all(item[-1][-3:] == "r")) and ((p[0], p[1]-1) not in item[1]) :
        
        new = d[p[0], p[1]+1] + item[0]
        # finished
        if (p[0] == len(d)) and (p[1]+1 == len(d[0])):
            print(new)
            break
        else:
            q.append((new, item[1] + [(p[0], p[1]+1)], np.append(item[2], "r" )))


1
2
3
3
4
6
7
8
10
9
11
12
13
12
12
13
13
13
15
17
18
20
21
21
22
24
26
26
27
28
28
29
29
31
31
32
32
33
35
34
35
34
36
37
38
38
39
41
40
41
43
45
46
47
49
51
52
54
56
56
57
57
57
59
59
59
60
62
64
64
65
66
66
65
65
66
68
69
69
70
70
70
70
70
70
71
72
71
72
74
74
75
74
76
76
77
80
81
82
81
82
84
85
87
89
89
91
92
93
95
96
97
96
98
99
101
103
103
103
105
105
106
106
108
109
110
109
111
113
114
115
114
114
114
115
116
118
118
120
121
122
122
123
124
126
128
129
131
132
133
135
135
136
136
138
140
142
144
146
148
150
151
152
153
152
151
153
153
154
156
157
159
159
160
160
161
163
164
166
167
169
169
169
170
172
173
173
174
175
175
175
176
178
179
181
181
182
182
183
185


In [16]:
q

[(-114,
  551,
  [(0, 0),
   (1, 0),
   (1, 1),
   (2, 1),
   (2, 0),
   (2, 1),
   (3, 1),
   (3, 0),
   (3, 1),
   (4, 1),
   (4, 0),
   (5, 0),
   (5, 1),
   (6, 1),
   (6, 0),
   (7, 0),
   (8, 0),
   (9, 0),
   (9, 1),
   (10, 1),
   (10, 0),
   (10, 1),
   (11, 1),
   (11, 2),
   (10, 2),
   (9, 2),
   (8, 2),
   (8, 3),
   (7, 3),
   (7, 2),
   (7, 3),
   (6, 3),
   (5, 3),
   (5, 4),
   (4, 4),
   (3, 4),
   (2, 4),
   (2, 5),
   (1, 5),
   (0, 5),
   (0, 6),
   (1, 6),
   (2, 6),
   (3, 6),
   (3, 5),
   (4, 5),
   (5, 5),
   (6, 5),
   (6, 6),
   (7, 6),
   (7, 5),
   (8, 5),
   (8, 6),
   (9, 6),
   (10, 6),
   (10, 5),
   (10, 6),
   (11, 6),
   (11, 7),
   (12, 7),
   (12, 8),
   (11, 8),
   (10, 8),
   (10, 9),
   (9, 9),
   (9, 10),
   (10, 10),
   (11, 10),
   (11, 11),
   (12, 11),
   (12, 12),
   (11, 12),
   (10, 12),
   (9, 12),
   (9, 11),
   (8, 11),
   (7, 11),
   (7, 12),
   (6, 12),
   (5, 12),
   (5, 11),
   (4, 11),
   (3, 11),
   (3, 12),
   (2, 12),
   (1, 